# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from numpy.linalg import matrix_power
from classes import distribution

# Bell Scenario Definition

In [2]:
N = 2

NA = N
NB = N
NX = N
NM = N

bell_scenario=(NA,NB,NX,NM)

A = np.array(range(NA))
X = np.array(range(NX))
B = np.array(range(NB))
M = np.array(range(NM))

# Distribution generator

for this particular one, we pick the optimal non-signaling distribution (not a quantum one) for the CHSH game.
$$
    p(a,b|x,m)= \frac{1}{2}\mathbb{1}_{a\oplus b = x\cdot m}
$$

In [6]:
vals = np.zeros(shape=(NA,NB,NX,NM))
for x in X:
    for m in M:
        for a in A:
            for b in B:
                lhs = (a+b)%2
                rhs = x*m
                val = 0.5*(lhs==rhs)
                vals[a,b,x,m] = val
p = distribution(vals, bell_scenario)

In [116]:
vals = np.zeros(shape=(NA,NB,NX,NM))
for x in X:
    for m in M:
        for a in A:
            for b in B:
                vals[a,b,x,m] = (a==x)*(b==m)
                vals[a,b,x,m] = 0.25
p = distribution(vals, bell_scenario)

For the first step of the NPA hierarchy, the $\Gamma$ matrix has the following form.
$$\Gamma\left(\mathcal{P}\right)=\left(\begin{array}{ccccccccc}
\overset{\left(\perp\right)}{\overbrace{1}} & \overset{\left(a=0,x=0\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(a=1,x=0\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(a=0,x=1\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(a=1,x=1\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(b=0,m=0\right)}{\overbrace{\Pr\left(b|m\right)}} & \overset{\left(b=1,m=0\right)}{\overbrace{\Pr\left(b|m\right)}} & \overset{\left(b=0,m=1\right)}{\overbrace{\Pr\left(b|m\right)}} & \overset{\left(b=1,m=1\right)}{\overbrace{\Pr\left(b|m\right)}}\\
 & \Pr\left(a|x\right) & 0 & ? & ? & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  & \Pr\left(a|x\right) & ? & ? & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  &  & \Pr\left(a|x\right) & 0 & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  &  &  & \Pr\left(a|x\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  &  &  &  & \Pr\left(b|m\right) & 0 & ? & ?\\
 &  &  &  &  &  & \Pr\left(b|m\right) & ? & ?\\
 &  &  &  &  &  &  & \Pr\left(b|m\right) & 0\\
 &  &  &  &  &  &  &  & \Pr\left(b|m\right)
\end{array}\right)$$

In [117]:
mu_vec = [cp.Variable() for _ in range(8)]
mu_gamma = cp.Variable()

In [118]:
Γ = cp.bmat([[1, p.marginal(0,0,0), p.marginal(1,0,0), p.marginal(0,1,0), p.marginal(1,1,0), p.marginal(0,0,0), p.marginal(1,0,0), p.marginal(0,1,0), p.marginal(1,1,0)]
            ,[p.marginal(0,0,0), p.marginal(0,0,0), 0, mu_vec[0], mu_vec[1], p(0,0,0,0), p(0,1,0,0), p(0,0,0,1), p(0,1,0,1)]
            ,[p.marginal(1,0,0), 0, p.marginal(1,0,0), mu_vec[2], mu_vec[3], p(1,0,0,0), p(1,1,0,0), p(1,0,0,1), p(1,1,0,1)]
            ,[p.marginal(0,1,0), mu_vec[0], mu_vec[2], p.marginal(0,1,0), 0, p(0,0,1,0), p(0,1,1,0), p(0,0,1,1), p(0,1,1,1)]
            ,[p.marginal(1,1,0), mu_vec[1], mu_vec[3], 0, p.marginal(1,1,0), p.marginal(0,0,0), p.marginal(1,0,0), p.marginal(0,1,0), p.marginal(1,1,0)]
            ,[p.marginal(0,0,1), p(0,0,0,0), p(1,0,0,0), p(0,0,1,0), p(1,0,1,0), p.marginal(0,0,1), 0, mu_vec[4], mu_vec[5]]
            ,[p.marginal(1,0,1), p(0,1,0,0), p(1,1,0,0), p(0,1,1,0), p(1,1,1,0), 0, p.marginal(1,0,1), mu_vec[6], mu_vec[7]]
            ,[p.marginal(0,1,1), p(0,0,0,1), p(1,0,0,1), p(0,0,1,1), p(1,0,1,1), mu_vec[4], mu_vec[6], p.marginal(0,1,1), 0]
            ,[p.marginal(1,1,1), p(0,1,0,1), p(1,1,0,1), p(0,1,1,1), p(1,1,1,1), mu_vec[5], mu_vec[7], 0, p.marginal(1,1,1)]  ])
Γ.value

In [119]:
constraints = list()
# constraints = [mu >= 0 for mu in mu_vec]
constraints += [Γ >> mu_gamma, Γ == Γ.T]

In [120]:
objective = cp.Maximize(mu_gamma)
problem = cp.Problem(objective, constraints)
problem.solve()


-inf

In [21]:
zero = np.array([[1,0]])
zero = zero.transpose() @ zero

plus = (2**(-0.5))*np.array([[1,1]])
plus = plus.transpose() @ plus


Π1 = cp.bmat([[1,0],[0,0]])
Π2 = cp.bmat([[1,0],[0,0]])
ρ = cp.Variable((2,2), symmetric=True)
objective = cp.Maximize(cp.trace(ρ @ Π))
constraints = [ρ >> 0, ρ << 1]

In [14]:
problem = cp.Problem(objective, constraints)
problem.solve()

0.9999997833762444

In [77]:
id = sigma_z @ sigma_z
sigma_z = np.array([[1,0],[0,-1]])
sigma_x = np.array([[0,1],[1,0]])

sigma_plus = (sigma_z + sigma_x) / (2**0.5)
sigma_minus = (sigma_z - sigma_x) / (2**0.5)

S = np.kron(sigma_z, sigma_plus)
S += np.kron(sigma_x, sigma_plus)
S += np.kron(sigma_z, sigma_minus)
S += -np.kron(sigma_x, sigma_minus)
S /= 8
S += np.kron(id, id) / 2

rho = cp.Variable((4,4), symmetric=True)
objective = cp.Maximize(cp.trace(rho @ S))
constraints = [rho >> 0, cp.trace(rho) <= 1]

problem = cp.Problem(objective, constraints)
problem.solve()

0.8535530361400055

In [84]:
id = sigma_z @ sigma_z
sigma_z = np.array([[1,0],[0,-1]])
sigma_x = np.array([[0,1],[1,0]])

sigma_plus = (sigma_z + sigma_x) / (2**0.5)
sigma_minus = (sigma_z - sigma_x) / (2**0.5)

S = np.kron(sigma_z, sigma_plus)
S += np.kron(sigma_x, sigma_plus)
S += np.kron(sigma_z, sigma_minus)
S += -np.kron(sigma_x, sigma_minus)
S /= 8
S += np.kron(id, id) / 2

gamma = [[cp.Variable() for _ in range(4)] for __ in range(4)]
Gamma = cp.bmat([[gamma[0][0], gamma[0][1], gamma[0][2], gamma[0][3]],
                 [gamma[0][1], gamma[1][1], gamma[1][2], gamma[1][3]],
                 [gamma[0][2], gamma[1][2], gamma[2][2], gamma[2][3]],
                 [gamma[0][3], gamma[1][3], gamma[2][3], gamma[3][3]]])

objective = cp.Maximize(cp.trace(Gamma @ S))
constraints = [Gamma >> 0, cp.trace(Gamma) <= 1, Gamma == Gamma.T]

problem = cp.Problem(objective, constraints)
problem.solve()

0.8535530361400002

In [112]:
gammas = [[cp.Variable() for _ in range(4)] for _ in range(4)]

In [113]:
gammas

[[Variable(()), Variable(()), Variable(()), Variable(())],
 [Variable(()), Variable(()), Variable(()), Variable(())],
 [Variable(()), Variable(()), Variable(()), Variable(())],
 [Variable(()), Variable(()), Variable(()), Variable(())]]